# အချိန်တပြိုင်နက် စီမံခန့်ခွဲမှုဖြင့် ခရီးသွားအကြံပြုချက်များ

ဤ notebook တွင် Microsoft Agent Framework ကို အသုံးပြု၍ **အချိန်တပြိုင်နက် စီမံခန့်ခွဲမှု** ကို ပြသထားသည်။ ခရီးသွားအကြံပြုချက် စနစ်တစ်ခုကို သီးခြားအထူးပြုထားသောအေးဂျင့်သုံးယောက်ဖြင့် 병렬စွာဆောင်ရွက်ပြီး စုံလင်သော ခရီးသွားအမြင်များကို ပံ့ပိုးပေးမည်ဖြစ်သည်။

## သင်ယူရမည့်အချက်များ:
1. **အချိန်တပြိုင်နက် စီမံခန့်ခွဲမှု**: အေးဂျင့်များစွာကို 병렬ဆောင်ရွက်ခြင်း (fan-out/fan-in ပုံစံ)
2. **ConcurrentBuilder**: အချိန်တပြိုင်နက် workflow များ တည်ဆောက်ရန် အဆင့်မြင့် API
3. **ခရီးသွားအကြံပြုချက်များ**: သီးခြားအထူးပြုထားသော အေးဂျင့်သုံးယောက်ပူးပေါင်းဆောင်ရွက်ခြင်း
4. **ပုံမှန်စုစုပေါင်းခြင်း**: အေးဂျင့်တချို့စာများကို ပေါင်းစည်းခြင်း
5. **စွမ်းဆောင်ရည်အကျိုးများ**: 병렬အလျင် vs အစဉ်လိုက်ကိုင်တွယ်မှု

## သီးခြားအထူးပြုထားသောအေးဂျင့်သုံးယောက်:

1. **Attractions Agent**: ပြည်တွင်းခရီးသွားဆွဲဆောင်မှုများ၊ လှုပ်ရှားမှုများ၊ အမှတ်တရများ
2. **Dining Agent**: ရိုးရာစားမည့်အစားအစာများ၊ ဟိုတယ်များ၊ အစားအစာအတွေ့အကြုံများ
3. **History Agent**: သမိုင်းအချက်အလက်များ၊ ယဉ်ကျေးမှုအရေးပါမှု၊ ပတ်ဝန်းကျင်


In [3]:
import asyncio
import json
import os
from typing import Any, cast

from agent_framework import (
    ChatMessage,
    ConcurrentBuilder,
    WorkflowOutputEvent,
)

# GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("All imports successful!")

All imports successful!


## Step 1: တွဲဖက်ထုတ်လွှင့်မှုများအတွက် Pydantic မော်ဒယ်များကို သတ်မှတ်ပါ

ဤမော်ဒယ်များသည် တစ်ဦးချင်းစီ အထူးပြု<|vq_image_9427|><|vq_image_3980|><|vq_image_13949|><|vq_image_4910|><|vq_image_14167|><|vq_image_12737|><|vq_image_7986|><|vq_image_10889|><|vq_image_14678|><|vq_image_11817|><|vq_image_11831|><|vq_image_9753|><|vq_image_5573|><|vq_image_5707|><|vq_image_8522|><|vq_image_15486|><|vq_image_8522|><|vq_image_5573|><|vq_image_2525|><|vq_image_1073|><|vq_image_8789|><|vq_image_4293|><|vq_image_15689|><|vq_image_13846|><|vq_image_7986|><|vq_image_522|><|vq_image_7591|><|vq_image_14167|><|vq_image_16158|><|vq_image_2294|><|vq_image_522|><|vq_image_13949|><|vq_image_7986|><|vq_image_14167|><|vq_image_7246|><|image_border_300|><|vq_image_10202|><|vq_image_813|><|vq_image_1615|><|vq_image_10655|><|vq_image_3118|><|vq_image_759|><|vq_image_4621|><|vq_image_16262|><|vq_image_7500|><|vq_image_4621|><|vq_image_9736|><|vq_image_1073|><|vq_image_16158|><|vq_image_1615|><|vq_image_16158|><|vq_image_8522|><|vq_image_11554|><|vq_image_15874|><|vq_image_4701|><|vq_image_7426|><|vq_image_9230|><|vq_image_12694|><|vq_image_13941|><|vq_image_759|><|vq_image_3118|><|vq_image_3118|><|vq_image_6846|><|vq_image_11831|><|vq_image_9569|><|vq_image_14793|><|vq_image_9230|><|vq_image_4293|><|vq_image_9497|><|vq_image_8522|><|vq_image_8574|><|vq_image_12248|><|vq_image_7986|><|vq_image_8743|><|vq_image_5459|><|vq_image_1937|><|vq_image_8821|><|vq_image_78|><|vq_image_8522|><|vq_image_7986|><|vq_image_16262|><|vq_image_7500|><|vq_image_12737|><|vq_image_14167|><|vq_image_3655|><|vq_image_7986|><|vq_image_12737|><|vq_image_10145|><|vq_image_8522|><|vq_image_10145|><|vq_image_2525|><|vq_image_5143|><|vq_image_10621|><|vq_image_4813|><|vq_image_7986|><|vq_image_13846|><|vq_image_12248|><|vq_image_9753|><|vq_image_7591|><|vq_image_3587|><|image_border_301|><|vq_image_7986|><|vq_image_1238|><|vq_image_7986|><|vq_image_10889|><|vq_image_8522|><|vq_image_8522|><|vq_image_4560|><|vq_image_8522|><|vq_image_7986|><|vq_image_16262|><|vq_image_7986|><|vq_image_4910|><|vq_image_1618|><|vq_image_10889|><|vq_image_8522|><|vq_image_8743|><|vq_image_7500|><|vq_image_7500|><|vq_image_8522|><|vq_image_10889|><|vq_image_13266|><|vq_image_15689|><|vq_image_2525|><|vq_image_13028|><|vq_image_15486|><|vq_image_7986|><|vq_image_6324|><|vq_image_6866|><|vq_image_15486|><|vq_image_6324|><|vq_image_11831|><|vq_image_11817|><|vq_image_7986|><|vq_image_11831|><|vq_image_15689|><|vq_image_8522|><|vq_image_12737|><|vq_image_4701|><|vq_image_3587|><|vq_image_11831|><|vq_image_9569|><|vq_image_12123|><|vq_image_13129|><|vq_image_3118|><|vq_image_7986|><|vq_image_15689|><|vq_image_12737|><|vq_image_16158|><|vq_image_4293|><|vq_image_244|><|vq_image_11831|><|vq_image_15059|><|vq_image_552|><|vq_image_5113|><|vq_image_9753|><|vq_image_1558|><|vq_image_2906|><|vq_image_5609|><|vq_image_7986|><|vq_image_1558|><|vq_image_3118|><|vq_image_7986|><|vq_image_8522|><|vq_image_2901|><|image_border_302|><|vq_image_12737|><|vq_image_15689|><|vq_image_10889|><|vq_image_10621|><|vq_image_5573|><|vq_image_5573|><|vq_image_1832|><|vq_image_5573|><|vq_image_8522|><|vq_image_5573|><|vq_image_7426|><|vq_image_4560|><|vq_image_10621|><|vq_image_8522|><|vq_image_10621|><|vq_image_1204|><|vq_image_3655|><|vq_image_7986|><|vq_image_8522|><|vq_image_8522|><|vq_image_5573|><|vq_image_2525|><|vq_image_15689|><|vq_image_7426|><|vq_image_9497|><|vq_image_7986|><|vq_image_15689|><|vq_image_15689|><|vq_image_7986|><|vq_image_7426|><|vq_image_3655|><|vq_image_11831|><|vq_image_2901|><|vq_image_5707|><|vq_image_11831|><|vq_image_7986|><|vq_image_8522|><|vq_image_2525|><|vq_image_3655|><|vq_image_15689|><|vq_image_9569|><|vq_image_7551|><|vq_image_11831|><|vq_image_5573|><|vq_image_1618|><|vq_image_10889|><|vq_image_10889|><|vq_image_7426|><|vq_image_1758|><|vq_image_78|><|vq_image_5573|><|vq_image_2525|><|vq_image_5573|><|vq_image_1558|><|vq_image_8522|><|vq_image_10889|><|vq_image_7986|><|vq_image_12737|><|vq_image_13523|><|vq_image_5459|><|vq_image_4910|><|vq_image_9569|><|vq_image_6603|><|vq_image_11817|><|image_border_303|><|vq_image_9753|><|vq_image_12737|><|vq_image_6355|><|vq_image_8522|><|vq_image_9753|><|vq_image_7426|><|vq_image_4293|><|vq_image_1558|><|vq_image_15689|><|vq_image_7986|><|vq_image_2525|><|vq_image_7986|><|vq_image_8522|><|vq_image_10889|><|vq_image_4293|><|vq_image_7426|><|vq_image_8522|><|vq_image_10889|><|vq_image_8522|><|vq_image_15689|><|vq_image_10889|><|vq_image_11831|><|vq_image_5481|><|vq_image_1984|><|vq_image_5573|><|vq_image_9753|><|vq_image_8789|><|vq_image_5707|><|vq_image_9736|><|vq_image_8522|><|vq_image_5573|><|vq_image_5573|><|vq_image_4293|><|vq_image_10889|><|vq_image_15689|><|vq_image_10889|><|vq_image_8522|><|vq_image_4293|><|vq_image_10889|><|vq_image_10889|><|vq_image_8522|><|vq_image_2578|><|vq_image_78|><|vq_image_7591|><|vq_image_5386|><|vq_image_10889|><|vq_image_802|><|vq_image_1558|><|vq_image_10889|><|vq_image_728|><|vq_image_1954|><|vq_image_10889|><|vq_image_4293|><|vq_image_9753|><|vq_image_5573|><|vq_image_10621|><|vq_image_10889|><|vq_image_10889|><|vq_image_10889|><|vq_image_759|><|vq_image_7591|><|vq_image_10621|><|vq_image_4293|><|vq_image_2901|><|image_border_304|><|vq_image_2540|><|vq_image_9763|><|vq_image_10869|><|vq_image_5872|><|vq_image_16262|><|vq_image_4560|><|vq_image_4560|><|vq_image_9763|><|vq_image_16262|><|vq_image_16262|><|vq_image_16262|><|vq_image_14967|><|vq_image_9763|><|vq_image_2550|><|vq_image_15689|><|vq_image_12737|><|vq_image_11818|><|vq_image_802|><|vq_image_12737|><|vq_image_12737|><|vq_image_12737|><|vq_image_13674|><|vq_image_13878|><|vq_image_7602|><|vq_image_8522|><|vq_image_8522|><|vq_image_8789|><|vq_image_11831|><|vq_image_4701|><|vq_image_8522|><|vq_image_4560|><|vq_image_8743|><|vq_image_4560|><|vq_image_289|><|vq_image_16262|><|vq_image_2550|><|vq_image_1558|><|vq_image_8522|><|vq_image_4701|><|vq_image_4560|><|vq_image_16206|><|vq_image_16206|><|vq_image_4701|><|vq_image_4701|><|vq_image_4701|><|vq_image_4560|><|vq_image_4701|><|vq_image_4701|><|vq_image_4701|><|vq_image_4701|><|vq_image_4701|><|vq_image_4701|><|vq_image_16206|><|vq_image_16206|><|vq_image_16206|><|vq_image_4701|><|vq_image_16262|><|vq_image_4701|><|vq_image_16206|><|vq_image_4701|><|vq_image_4701|><|vq_image_14967|><|vq_image_4701|><|vq_image_9753|><|image_border_305|><|vq_image_8375|><|vq_image_16206|><|vq_image_16144|><|vq_image_3679|><|vq_image_5972|><|vq_image_1659|><|vq_image_1659|><|vq_image_3803|><|vq_image_4507|><|vq_image_12142|><|vq_image_14050|><|vq_image_3803|><|vq_image_16206|><|vq_image_5682|><|vq_image_6070|><|vq_image_16368|><|vq_image_1659|><|vq_image_871|><|vq_image_1937|><|vq_image_1659|><|vq_image_14167|><|vq_image_14967|><|vq_image_5898|><|vq_image_8375|><|vq_image_12142|><|vq_image_14967|><|vq_image_16206|><|vq_image_8279|><|vq_image_14967|><|vq_image_9763|><|vq_image_8026|><|vq_image_15589|><|vq_image_9763|><|vq_image_9763|><|vq_image_4701|><|vq_image_4701|><|vq_image_12142|><|vq_image_12142|><|vq_image_12142|><|vq_image_12142|><|vq_image_12142|><|vq_image_8279|><|vq_image_12142|><|vq_image_12142|><|vq_image_4836|><|vq_image_12142|><|vq_image_12142|><|vq_image_12142|><|vq_image_12142|><|vq_image_12142|><|vq_image_4836|><|vq_image_12142|><|vq_image_4836|><|vq_image_4836|><|vq_image_14050|><|vq_image_14050|><|vq_image_12142|><|vq_image_14050|><|vq_image_12142|><|vq_image_12142|><|vq_image_14050|><|vq_image_14050|><|vq_image_12142|><|vq_image_12142|><|image_border_306|><|vq_image_4836|><|vq_image_13543|><|vq_image_6648|><|vq_image_13493|><|vq_image_13493|><|vq_image_14388|><|vq_image_4836|><|vq_image_4701|><|vq_image_4701|><|vq_image_4701|><|vq_image_14050|><|vq_image_6104|><|vq_image_4836|><|vq_image_8743|><|vq_image_7591|><|vq_image_16206|><|vq_image_4701|><|vq_image_1792|><|vq_image_3803|><|vq_image_14967|><|vq_image_14388|><|vq_image_13543|><|vq_image_7347|><|vq_image_3743|><|vq_image_9655|><|vq_image_1674|><|vq_image_4202|><|vq_image_13407|><|vq_image_7223|><|vq_image_16144|><|vq_image_8873|><|vq_image_1674|><|vq_image_6351|><|vq_image_16144|><|vq_image_16144|><|vq_image_16144|><|vq_image_16144|><|vq_image_16144|><|vq_image_16144|><|vq_image_16144|><|vq_image_16144|><|vq_image_1674|><|vq_image_1674|><|vq_image_1674|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_14050|><|vq_image_3803|><|vq_image_3803|><|vq_image_3803|><|vq_image_6966|><|vq_image_14946|><|vq_image_4836|><|vq_image_4836|><|image_border_307|><|vq_image_13523|><|vq_image_8522|><|vq_image_5707|><|vq_image_10889|><|vq_image_5573|><|vq_image_10889|><|vq_image_9753|><|vq_image_5573|><|vq_image_5573|><|vq_image_5707|><|vq_image_8522|><|vq_image_9753|><|vq_image_2525|><|vq_image_2525|><|vq_image_15689|><|vq_image_9250|><|vq_image_10889|><|vq_image_12737|><|vq_image_9753|><|vq_image_2525|><|vq_image_1204|><|vq_image_13410|><|vq_image_1461|><|vq_image_6035|><|vq_image_1204|><|vq_image_11040|><|vq_image_6234|><|vq_image_6869|><|vq_image_9339|><|vq_image_9043|><|vq_image_759|><|vq_image_12928|><|vq_image_1058|><|vq_image_12928|><|vq_image_759|><|vq_image_2525|><|vq_image_12928|><|vq_image_15486|><|vq_image_9753|><|vq_image_2525|><|vq_image_335|><|vq_image_3913|><|vq_image_1204|><|vq_image_1204|><|vq_image_1204|><|vq_image_7552|><|vq_image_1204|><|vq_image_7085|><|vq_image_1204|><|vq_image_1204|><|vq_image_15689|><|vq_image_1549|><|vq_image_4639|><|vq_image_13028|><|vq_image_9753|><|vq_image_1204|><|vq_image_7551|><|vq_image_1204|><|vq_image_1549|><|vq_image_1204|><|vq_image_1204|><|vq_image_12928|><|vq_image_335|><|vq_image_4910|><|image_border_308|><|vq_image_5280|><|vq_image_13028|><|vq_image_8821|><|vq_image_2332|><|vq_image_11817|><|vq_image_48|><|vq_image_562|><|vq_image_15486|><|vq_image_11866|><|vq_image_11512|><|vq_image_48|><|vq_image_15831|><|vq_image_2076|><|vq_image_15328|><|vq_image_7085|><|vq_image_9457|><|vq_image_562|><|vq_image_13675|><|vq_image_4672|><|vq_image_15037|><|vq_image_2076|><|vq_image_12073|><|vq_image_7671|><|vq_image_12694|><|vq_image_8962|><|vq_image_2246|><|vq_image_1159|><|vq_image_4348|><|vq_image_6|><|vq_image_728|><|vq_image_11353|><|vq_image_2132|><|vq_image_4240|><|vq_image_4621|><|vq_image_1159|><|vq_image_14581|><|vq_image_13818|><|vq_image_16194|><|vq_image_7633|><|vq_image_11554|><|vq_image_11591|><|vq_image_13809|><|vq_image_7986|><|vq_image_5332|><|vq_image_2046|><|vq_image_13806|><|vq_image_1159|><|vq_image_728|><|vq_image_7906|><|vq_image_6648|><|vq_image_1159|><|vq_image_534|><|vq_image_9192|><|vq_image_4408|><|vq_image_11554|><|vq_image_1777|><|vq_image_11554|><|vq_image_1159|><|vq_image_11554|><|vq_image_728|><|vq_image_7452|><|vq_image_10923|><|vq_image_10923|><|vq_image_16157|><|image_border_309|><|vq_image_3752|><|vq_image_15068|><|vq_image_15093|><|vq_image_11558|><|vq_image_13557|><|vq_image_4625|><|vq_image_13493|><|vq_image_12965|><|vq_image_11591|><|vq_image_14636|><|vq_image_4190|><|vq_image_10745|><|vq_image_15531|><|vq_image_5713|><|vq_image_8958|><|vq_image_13016|><|vq_image_7823|><|vq_image_11591|><|vq_image_15450|><|vq_image_12639|><|vq_image_5750|><|vq_image_16197|><|vq_image_6966|><|vq_image_13060|><|vq_image_11912|><|vq_image_10244|><|vq_image_4190|><|vq_image_85|><|vq_image_11831|><|vq_image_4437|><|vq_image_14050|><|vq_image_2578|><|vq_image_12392|><|vq_image_5750|><|vq_image_3874|><|vq_image_9910|><|vq_image_5750|><|vq_image_14050|><|vq_image_15152|><|vq_image_14547|><|vq_image_3752|><|vq_image_11138|><|vq_image_7406|><|vq_image_7402|><|vq_image_10291|><|vq_image_5750|><|vq_image_11558|><|vq_image_6966|><|vq_image_11558|><|vq_image_11558|><|vq_image_11558|><|vq_image_5750|><|vq_image_15589|><|vq_image_7277|><|vq_image_1073|><|vq_image_7494|><|vq_image_11558|><|vq_image_5750|><|vq_image_11558|><|vq_image_5750|><|vq_image_7452|><|vq_image_1674|><|vq_image_5750|><|vq_image_15589|><|image_border_310|><|vq_image_12082|><|vq_image_7223|><|vq_image_2492|><|vq_image_2094|><|vq_image_709|><|vq_image_12160|><|vq_image_2632|><|vq_image_4937|><|vq_image_2632|><|vq_image_14633|><|vq_image_8478|><|vq_image_2308|><|vq_image_11821|><|vq_image_14478|><|vq_image_11267|><|vq_image_13062|><|vq_image_16284|><|vq_image_12648|><|vq_image_10101|><|vq_image_812|><|vq_image_1729|><|vq_image_8518|><|vq_image_14371|><|vq_image_6922|><|vq_image_10166|><|vq_image_701|><|vq_image_6342|><|vq_image_3876|><|vq_image_701|><|vq_image_3876|><|vq_image_3821|><|vq_image_1498|><|vq_image_13587|><|vq_image_7276|><|vq_image_12693|><|vq_image_11821|><|vq_image_6774|><|vq_image_5402|><|vq_image_5402|><|vq_image_11924|><|vq_image_5402|><|vq_image_9962|><|vq_image_15135|><|vq_image_1498|><|vq_image_11821|><|vq_image_1498|><|vq_image_3876|><|vq_image_8483|><|vq_image_1498|><|vq_image_1498|><|vq_image_1498|><|vq_image_1498|><|vq_image_1498|><|vq_image_9738|><|vq_image_13252|><|vq_image_9738|><|vq_image_9738|><|vq_image_9738|><|vq_image_15135|><|vq_image_9738|><|vq_image_9738|><|vq_image_9738|><|vq_image_9738|><|vq_image_9738|><|image_border_311|><|vq_image_11614|><|vq_image_7724|><|vq_image_6944|><|vq_image_4573|><|vq_image_3233|><|vq_image_979|><|vq_image_7889|><|vq_image_12197|><|vq_image_3762|><|vq_image_12497|><|vq_image_12674|><|vq_image_9292|><|vq_image_8967|><|vq_image_85|><|vq_image_2217|><|vq_image_13918|><|vq_image_1788|><|vq_image_16345|><|vq_image_13561|><|vq_image_232|><|vq_image_2402|><|vq_image_14468|><|vq_image_3014|><|vq_image_2678|><|vq_image_3331|><|vq_image_3305|><|vq_image_5698|><|vq_image_13060|><|vq_image_14440|><|vq_image_5894|><|vq_image_14742|><|vq_image_2437|><|vq_image_11426|><|vq_image_12630|><|vq_image_9198|><|vq_image_4507|><|vq_image_16013|><|vq_image_5972|><|vq_image_14221|><|vq_image_979|><|vq_image_14742|><|vq_image_15656|><|vq_image_4190|><|vq_image_5897|><|vq_image_9299|><|vq_image_305|><|vq_image_376|><|vq_image_11068|><|vq_image_6104|><|vq_image_8480|><|vq_image_14678|><|vq_image_10805|><|vq_image_15959|><|vq_image_12860|><|vq_image_4645|><|vq_image_13031|><|vq_image_7764|><|vq_image_8927|><|vq_image_6885|><|vq_image_3305|><|vq_image_14388|><|vq_image_13557|><|vq_image_8241|><|vq_image_979|><|image_border_312|><|vq_image_941|><|vq_image_5776|><|vq_image_3335|><|vq_image_6132|><|vq_image_9062|><|vq_image_8763|><|vq_image_5246|><|vq_image_1033|><|vq_image_2958|><|vq_image_14331|><|vq_image_13543|><|vq_image_14359|><|vq_image_12939|><|vq_image_10462|><|vq_image_8823|><|vq_image_4163|><|vq_image_14252|><|vq_image_8798|><|vq_image_5257|><|vq_image_8241|><|vq_image_1597|><|vq_image_14050|><|vq_image_7143|><|vq_image_8407|><|vq_image_14147|><|vq_image_13543|><|vq_image_9307|><|vq_image_5727|><|vq_image_15208|><|vq_image_13543|><|vq_image_7223|><|vq_image_4308|><|vq_image_9017|><|vq_image_13728|><|vq_image_9460|><|vq_image_4261|><|vq_image_495|><|vq_image_11286|><|vq_image_1774|><|vq_image_1878|><|vq_image_15589|><|vq_image_93|><|vq_image_12125|><|vq_image_3571|><|vq_image_14443|><|vq_image_8147|><|vq_image_1438|><|vq_image_3517|><|vq_image_4980|><|vq_image_4253|><|vq_image_14257|><|vq_image_11614|><|vq_image_2726|><|vq_image_11054|><|vq_image_6282|><|vq_image_4419|><|vq_image_6349|><|vq_image_13112|><|vq_image_15617|><|vq_image_3709|><|vq_image_3129|><|vq_image_1087|><|vq_image_5369|><|vq_image_9818|><|image_border_313|><|vq_image_13316|><|vq_image_2148|><|vq_image_11175|><|vq_image_15321|><|vq_image_16340|><|vq_image_13745|><|vq_image_13031|><|vq_image_13377|><|vq_image_2704|><|vq_image_8530|><|vq_image_4163|><|vq_image_8544|><|vq_image_8969|><|vq_image_5750|><|vq_image_495|><|vq_image_16235|><|vq_image_9502|><|vq_image_10280|><|vq_image_272|><|vq_image_12084|><|vq_image_4941|><|vq_image_16007|><|vq_image_7047|><|vq_image_6788|><|vq_image_6885|><|vq_image_1225|><|vq_image_14361|><|vq_image_3001|><|vq_image_93|><|vq_image_8791|><|vq_image_9882|><|vq_image_3834|><|vq_image_5684|><|vq_image_7223|><|vq_image_3834|><|vq_image_12184|><|vq_image_11299|><|vq_image_13090|><|vq_image_4549|><|vq_image_12392|><|vq_image_13734|><|vq_image_14085|><|vq_image_686|><|vq_image_13794|><|vq_image_8962|><|vq_image_10687|><|vq_image_12084|><|vq_image_13940|><|vq_image_3874|><|vq_image_5652|><|vq_image_11110|><|vq_image_3818|><|vq_image_6251|><|vq_image_8530|><|vq_image_14156|><|vq_image_13607|><|vq_image_6887|><|vq_image_6887|><|vq_image_1379|><|vq_image_3633|><|vq_image_5768|><|vq_image_12080|><|vq_image_12242|><|vq_image_3633|><|image_border_314|><|vq_image_5713|><|vq_image_7408|><|vq_image_14547|><|vq_image_14018|><|vq_image_10291|><|vq_image_13934|><|vq_image_4704|><|vq_image_13734|><|vq_image_3571|><|vq_image_5611|><|vq_image_727|><|vq_image_1493|><|vq_image_495|><|vq_image_14085|><|vq_image_11867|><|vq_image_14742|><|vq_image_12543|><|vq_image_12639|><|vq_image_4739|><|vq_image_4739|><|vq_image_1073|><|vq_image_7402|><|vq_image_10879|><|vq_image_15605|><|vq_image_5573|><|vq_image_12184|><|vq_image_6250|><|vq_image_7389|><|vq_image_14488|><|vq_image_12084|><|vq_image_7126|><|vq_image_4197|><|vq_image_800|><|vq_image_9910|><|vq_image_10117|><|vq_image_3633|><|vq_image_917|><|vq_image_9655|><|vq_image_9910|><|vq_image_663|><|vq_image_10291|><|vq_image_5371|><|vq_image_16144|><|vq_image_9637|><|vq_image_6385|><|vq_image_12794|><|vq_image_5209|><|vq_image_6085|><|vq_image_69|><|vq_image_7618|><|vq_image_3571|><|vq_image_297|><|vq_image_6349|><|vq_image_11935|><|vq_image_3268|><|vq_image_7618|><|vq_image_3818|><|vq_image_128|><|vq_image_13031|><|vq_image_4625|><|vq_image_13745|><|vq_image_8209|><|vq_image_13254|><|vq_image_14257|><|image_border_315|><|vq_image_3248|><|vq_image_3721|><|vq_image_16244|><|vq_image_495|><|vq_image_14109|><|vq_image_11110|><|vq_image_7949|><|vq_image_10535|><|vq_image_8645|><|vq_image_4739|><|vq_image_3031|><|vq_image_6508|><|vq_image_4081|><|vq_image_15387|><|vq_image_2572|><|vq_image_3974|><|vq_image_16170|><|vq_image_6699|><|vq_image_15121|><|vq_image_85|><|vq_image_13589|><|vq_image_3281|><|vq_image_14318|><|vq_image_11052|><|vq_image_10566|><|vq_image_10405|><|vq_image_14490|><|vq_image_6104|><|vq_image_16262|><|vq_image_4854|><|vq_image_10291|><|vq_image_14050|><|vq_image_2442|><|vq_image_10921|><|vq_image_258|><|vq_image_15847|><|vq_image_10117|><|vq_image_4739|><|vq_image_13031|><|vq_image_2958|><|vq_image_2743|><|vq_image_10879|><|vq_image_8241|><|vq_image_11362|><|vq_image_11922|><|vq_image_4884|><|vq_image_4573|><|vq_image_12068|><|vq_image_7143|><|vq_image_85|><|vq_image_10115|><|vq_image_4674|><|vq_image_16206|><|vq_image_7889|><|vq_image_6568|><|vq_image_9981|><|vq_image_2509|><|vq_image_15769|><|vq_image_6162|><|vq_image_1087|><|vq_image_11886|><|vq_image_14453|><|vq_image_3281|><|vq_image_1087|><|image_border_316|><|vq_image_10033|><|vq_image_2801|><|vq_image_15516|><|vq_image_8997|><|vq_image_9327|><|vq_image_14514|><|vq_image_2422|><|vq_image_7717|><|vq_image_5347|><|vq_image_10522|><|vq_image_14344|><|vq_image_6468|><|vq_image_13561|><|vq_image_5180|><|vq_image_1966|><|vq_image_1510|><|vq_image_12204|><|vq_image_14546|><|vq_image_5057|><|vq_image_4875|><|vq_image_14124|><|vq_image_6648|><|vq_image_5236|><|vq_image_16097|><|vq_image_4713|><|vq_image_917|><|vq_image_4925|><|vq_image_14151|><|vq_image_1804|><|vq_image_520|><|vq_image_1628|><|vq_image_2862|><|vq_image_12124|><|vq_image_7931|><|vq_image_16105|><|vq_image_3537|><|vq_image_3531|><|vq_image_11591|><|vq_image_1345|><|vq_image_3461|><|vq_image_553|><|vq_image_5145|><|vq_image_5625|><|vq_image_7126|><|vq_image_15764|><|vq_image_4210|><|vq_image_11082|><|vq_image_14167|><|vq_image_10573|><|vq_image_15959|><|vq_image_7118|><|vq_image_2458|><|vq_image_258|><|vq_image_8299|><|vq_image_2862|><|vq_image_2538|><|vq_image_6966|><|vq_image_4549|><|vq_image_14889|><|vq_image_6169|><|vq_image_7949|><|vq_image_7401|><|vq_image_7401|><|vq_image_14547|><|image_border_317|><|vq_image_15068|><|vq_image_8851|><|vq_image_10054|><|vq_image_6667|><|vq_image_13637|><|vq_image_10125|><|vq_image_280|><|vq_image_5347|><|vq_image_2188|><|vq_image_15764|><|vq_image_5473|><|vq_image_15503|><|vq_image_7401|><|vq_image_6349|><|vq_image_12639|><|vq_image_935|><|vq_image_3627|><|vq_image_492|><|vq_image_12554|><|vq_image_13060|><|vq_image_7511|><|vq_image_8607|><|vq_image_15862|><|vq_image_11202|><|vq_image_13728|><|vq_image_1409|><|vq_image_5145|><|vq_image_12822|><|vq_image_4615|><|vq_image_12418|><|vq_image_4560|><|vq_image_10879|><|vq_image_2384|><|vq_image_11477|><|vq_image_13060|><|vq_image_11818|><|vq_image_12965|><|vq_image_93|><|vq_image_13628|><|vq_image_14371|><|vq_image_5692|><|vq_image_14488|><|vq_image_15222|><|vq_image_7020|><|vq_image_5750|><|vq_image_4674|><|vq_image_3709|><|vq_image_13053|><|vq_image_1729|><|vq_image_13648|><|vq_image_2437|><|vq_image_5735|><|vq_image_7171|><|vq_image_1087|><|vq_image_6869|><|vq_image_11731|><|vq_image_5145|><|vq_image_9664|><|vq_image_13031|><|vq_image_1324|><|vq_image_12008|><|vq_image_12402|><|vq_image_2509|><|vq_image_6568|><|image_border_318|><|vq_image_13031|><|vq_image_7195|><|vq_image_14488|><|vq_image_16028|><|vq_image_6953|><|vq_image_12451|><|vq_image_15772|><|vq_image_1666|><|vq_image_15004|><|vq_image_258|><|vq_image_8015|><|vq_image_4376|><|vq_image_10265|><|vq_image_9912|><|vq_image_14350|><|vq_image_7962|><|vq_image_95|><|vq_image_11557|><|vq_image_8629|><|vq_image_6387|><|vq_image_13409|><|vq_image_11857|><|vq_image_6040|><|vq_image_9589|><|vq_image_13214|><|vq_image_2046|><|vq_image_14488|><|vq_image_4723|><|vq_image_14488|><|vq_image_11041|><|vq_image_93|><|vq_image_10244|><|vq_image_4419|><|vq_image_1804|><|vq_image_9806|><|vq_image_8336|><|vq_image_14488|><|vq_image_13239|><|vq_image_4081|><|vq_image_15152|><|vq_image_4503|><|vq_image_10207|><|vq_image_3198|><|vq_image_14488|><|vq_image_10879|><|vq_image_9637|><|vq_image_5684|><|vq_image_5611|><|vq_image_10249|><|vq_image_11297|><|vq_image_15152|><|vq_image_13144|><|vq_image_353|><|vq_image_5587|><|vq_image_4625|><|vq_image_709|><|vq_image_9309|><|vq_image_9062|><|vq_image_536|><|vq_image_8015|><|vq_image_15543|><|vq_image_9570|><|vq_image_709|><|vq_image_15152|><|image_border_319|><|vq_image_5039|><|vq_image_13930|><|vq_image_7618|><|vq_image_7363|><|vq_image_2447|><|vq_image_7117|><|vq_image_9588|><|vq_image_12445|><|vq_image_15405|><|vq_image_13734|><|vq_image_6323|><|vq_image_9673|><|vq_image_11614|><|vq_image_12197|><|vq_image_1311|><|vq_image_6530|><|vq_image_6097|><|vq_image_12862|><|vq_image_9502|><|vq_image_15383|><|vq_image_15042|><|vq_image_16364|><|vq_image_9301|><|vq_image_9088|><|vq_image_10117|><|vq_image_8645|><|vq_image_14024|><|vq_image_7171|><|vq_image_8645|><|vq_image_2147|><|vq_image_8645|><|vq_image_10249|><|vq_image_13011|><|vq_image_5055|><|vq_image_10117|><|vq_image_2147|><|vq_image_8645|><|vq_image_5932|><|vq_image_2147|><|vq_image_2147|><|vq_image_5932|><|vq_image_4698|><|vq_image_2147|><|vq_image_5932|><|vq_image_6634|><|vq_image_15383|><|vq_image_10117|><|vq_image_2147|><|vq_image_8645|><|vq_image_10244|><|vq_image_8645|><|vq_image_8645|><|vq_image_12540|><|vq_image_93|><|vq_image_6634|><|vq_image_1674|><|vq_image_8645|><|vq_image_10117|><|vq_image_10249|><|vq_image_9901|><|vq_image_10117|><|vq_image_8645|><|vq_image_10879|><|vq_image_10249|>


## Step 1: ဖော်ပြထားသော Output များအတွက် Pydantic မော်ဒယ်များကို သတ်မှတ်ခြင်း

ဒီမော်ဒယ်တွေကတော့ အထူးပြုထားတဲ့ agent တစ်ယောက်စီက ပြန်ပေးတဲ့ schema ကို သတ်မှတ်ပေးပါတယ်။ ဒါက agent အားလုံးကနေ တူညီပြီး ပြန်ပေးချက်တွေကို အမှန်တကယ် ဖတ်ရှု့နိုင်စေရန် သေချာစေပါတယ်။


In [6]:
class AttractionsRecommendation(BaseModel):
    """Tourist attractions and activities recommendations."""

    destination: str
    top_attractions: list[str]
    activities: list[str]
    best_time_to_visit: str
    transportation_tips: str  


class DiningRecommendation(BaseModel):
    """Food and dining recommendations."""

    destination: str
    local_cuisine: str
    must_try_dishes: list[str]
    recommended_restaurants: list[str]
    food_experiences: list[str]
    dining_etiquette: str


class HistoryRecommendation(BaseModel):
    """Historical and cultural information."""

    destination: str
    historical_significance: str
    cultural_highlights: list[str]
    important_periods: list[str]
    cultural_experiences: list[str]
    interesting_facts: list[str]

## Step 2: ပတ်ဝန်းကျင်ချိန်ညှိမှုများ ကို ထည့်သွင်းရန်

LLM client ကို (GitHub Models သို့မဟုတ် OpenAI) middleware notebook နမူနာနှင့်တူညီသောပုံစံအတိုင်း စီစဉ်ပါ။


In [10]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"),
    api_key=os.environ.get("GITHUB_TOKEN"),
    model_id="gpt-4o"
)

## အဆင့် ၃: အထူးပြု ခရီးသွားတာဝန်ရှိ သုံးဦး တည်ဆောက်ပါ။


In [11]:
# Agent 1: Tourist Attractions Expert
attractions_agent = chat_client.create_agent(
    instructions=(
        "You are a tourism expert specializing in attractions and activities. "
        "When given a travel destination, provide comprehensive recommendations for "
        "tourist attractions, activities, best times to visit, and transportation tips. "
        "Focus on popular landmarks, unique experiences, and practical travel advice. "
        "Return structured JSON with the specified fields."
    ),
    name="attractions_agent",
    response_format=AttractionsRecommendation,
)

# Agent 2: Food and Dining Expert
dining_agent = chat_client.create_agent(
    instructions=(
        "You are a culinary expert specializing in local food and dining experiences. "
        "When given a travel destination, provide recommendations for local cuisine, "
        "must-try dishes, recommended restaurants, and unique food experiences. "
        "Include dining etiquette and cultural food customs. "
        "Return structured JSON with the specified fields."
    ),
    name="dining_agent",
    response_format=DiningRecommendation,
)


# Agent 3: History and Culture Expert
history_agent = chat_client.create_agent(
    instructions=(
        "You are a historian and cultural expert. "
        "When given a travel destination, provide historical context, cultural significance, "
        "important historical periods, cultural experiences, and interesting facts. "
        "Focus on helping travelers understand the cultural heritage and historical importance. "
        "Return structured JSON with the specified fields."
    ),
    name="history_agent",
    response_format=HistoryRecommendation,
)

# အဆင့် ၄: အချိန်တူ လုပ်ငန်းစဉ်ကို တည်ဆောက်ခြင်း

ConcurrentBuilder သည် အောက်ပါအတိုင်း လုပ်ငန်းစဉ်တစ်ခု ကို ဖန်တီးသည်။
1. **တစ်ပြိုင်နက်တည်း** တူညီသော အဝင်ကို အေးဂျင့်သုံးရပ်အားလုံးသို့ တပြိုင်နက်တည်း ပို့သည် (fan-out)
2. **အေးဂျင့်များကို** ထပ်တူပြိုင်ဆိုင်စွာ အကောင်းဆုံးဆောင်ရွက်နိုင်စေရန် ပြုလုပ်သည်
3. **တုံ့ပြန်ချက်များအားလုံးကို** တစ်ခုတည်းသော အထွက်သို့ စုစည်းသည် (fan-in)
4. **အေးဂျင့်အားလုံးမှ** ပေါင်းစပ်ထားသည့် ChatMessage စာရင်းကို ပြန်လည်ပေးပို့သည်


In [12]:
# Build the concurrent workflow using ConcurrentBuilder
workflow = (
    ConcurrentBuilder()
    .participants([attractions_agent, dining_agent, history_agent])
    .build()
)

display(HTML("""
<div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
    <h3 style='margin: 0 0 15px 0;'>Concurrent Workflow Built Successfully!</h3>
    <p style='margin: 0; line-height: 1.6;'>
        <strong>Architecture:</strong><br>
        • Input → <strong>Dispatcher</strong> (fan-out)<br>
        • <strong>3 Agents</strong> run in parallel (attractions, dining, history)<br>
        • <strong>Aggregator</strong> combines results (fan-in)<br>
        • Output → Combined travel recommendations
    </p>
</div>
"""))

## ခြေလှမ်း ၅: စမ်းသပ်မှု အမှု ၁ - တိုကျို ခရီးသွားအကြံပြုချက်များ

တိုကျိုကို ပန်းတိုင်ထား၍ ကျွန်ုပ်တို့၏ အချိန်တူလုပ်ဆောင်ရေး လုပ်ငန်းစဉ်ကို စမ်းသပ်ကြရအောင်။ သုံးဦးသော အေးဂျင့်များအားလုံးသည် တစ်ပြိုင်နက်မှာ အပြည့်အစုံသော ခရီးသွားအကြံပြုချက်များ ပေးစွမ်းရန် အတူတကွ အလုပ်လုပ်မည်ဖြစ်သည်။


In [1]:
async def display_travel_recommendations(destination: str):
    """Run the concurrent workflow and display formatted results."""

    display(HTML(f"""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Processing Travel Recommendations for {destination}</h3>
        <p style='margin: 0;'><strong>Status:</strong> Running 3 agents concurrently...</p>
    </div>
    """))

    # Run the workflow
    events = await workflow.run(f"I want comprehensive travel recommendations for {destination}")
    outputs = events.get_outputs()

    if outputs:
        # Get the aggregated messages from all agents
        messages: list[ChatMessage] = outputs[0]
        # Separate messages by agent (skip user message)
        agent_responses = [msg for msg in messages if msg.author_name in [
            "attractions_agent", "dining_agent", "history_agent"]]

        # Display results
        display(HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h2 style='margin: 0 0 20px 0;'>Complete Travel Guide for {destination}</h2>
            <p style='margin: 0; font-size: 14px; opacity: 0.9;'>Generated by 3 concurrent agents</p>
        </div>
        """))
        # Process each agent's response
        for msg in agent_responses:
            agent_name = msg.author_name

            try:
                # Parse the JSON response
                if agent_name == "attractions_agent":
                    data = AttractionsRecommendation.model_validate_json(
                        msg.text)
                    display_attractions_section(data)
                elif agent_name == "dining_agent":
                    data = DiningRecommendation.model_validate_json(msg.text)
                    display_dining_section(data)
                elif agent_name == "history_agent":
                    data = HistoryRecommendation.model_validate_json(msg.text)
                    display_history_section(data)
            except Exception as e:
                display(HTML(f"""
                <div style='padding: 15px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>Error parsing {agent_name} response:</strong> {str(e)}
                    <details><summary>Raw response</summary>{msg.text}</details>
                </div>
                """))
def display_attractions_section(data: AttractionsRecommendation):
    """Display attractions recommendations in a formatted section."""
    attractions_list = "".join([f"<li>{attraction}</li>" for attraction in data.top_attractions])
    activities_list = "".join([f"<li>{activity}</li>" for activity in data.activities])
    
    display(HTML(f"""
    <div style='padding: 20px; background: #e3f2fd; border-radius: 8px; margin: 15px 0; border-left: 4px solid #2196f3;'>
        <h3 style='margin: 0 0 15px 0; color: #1976d2;'>🏛️ Tourist Attractions & Activities</h3>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Top Attractions:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{attractions_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
        <h4 style='margin: 0 0 8px 0; color: #333;'>Recommended Activities:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{activities_list}</ul>
        </div>
        <div style='margin-bottom: 10px;'>
            <strong style='color: #333;'>Best Time to Visit:</strong> {data.best_time_to_visit}
        </div>
        <div>
            <strong style='color: #333;'>Transportation Tips:</strong> {data.transportation_tips}
        </div>
    </div>
    """))


def display_dining_section(data: DiningRecommendation):
    """Display dining recommendations in a formatted section."""
    dishes_list = "".join(
        [f"<li>{dish}</li>" for dish in data.must_try_dishes])
    restaurants_list = "".join(
        [f"<li>{restaurant}</li>" for restaurant in data.recommended_restaurants])
    experiences_list = "".join(
        [f"<li>{exp}</li>" for exp in data.food_experiences])

    display(HTML(f"""
    <div style='padding: 20px; background: #fff3e0; border-radius: 8px; margin: 15px 0; border-left: 4px solid #ff9800;'>
        <h3 style='margin: 0 0 15px 0; color: #f57c00;'>🍜 Food & Dining Experiences</h3>
        <div style='margin-bottom: 15px;'>
            <strong style='color: #333;'>Local Cuisine:</strong> {data.local_cuisine}
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Must-Try Dishes:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{dishes_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Recommended Restaurants:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{restaurants_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Food Experiences:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{experiences_list}</ul>
        </div>
        <div>
            <strong style='color: #333;'>Dining Etiquette:</strong> {data.dining_etiquette}
        </div>
    </div>
    """))


def display_history_section(data: HistoryRecommendation):
    """Display history recommendations in a formatted section."""
    highlights_list = "".join(
        [f"<li>{highlight}</li>" for highlight in data.cultural_highlights])
    periods_list = "".join(
        [f"<li>{period}</li>" for period in data.important_periods])
    experiences_list = "".join(
        [f"<li>{exp}</li>" for exp in data.cultural_experiences])
    facts_list = "".join(
        [f"<li>{fact}</li>" for fact in data.interesting_facts])

    display(HTML(f"""
    <div style='padding: 20px; background: #f3e5f5; border-radius: 8px; margin: 15px 0; border-left: 4px solid #9c27b0;'>
        <h3 style='margin: 0 0 15px 0; color: #7b1fa2;'>📚 History & Culture</h3>
        <div style='margin-bottom: 15px;'>
            <strong style='color: #333;'>Historical Significance:</strong> {data.historical_significance}
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Cultural Highlights:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{highlights_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Important Historical Periods:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{periods_list}</ul>
        </div>
        <div style='margin-bottom: 15px;'>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Cultural Experiences:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{experiences_list}</ul>
        </div>
        <div>
            <h4 style='margin: 0 0 8px 0; color: #333;'>Interesting Facts:</h4>
            <ul style='margin: 0; padding-left: 20px; color: #555;'>{facts_list}</ul>
        </div>
    </div>
    """))

    # Test with Tokyo
await display_travel_recommendations("Tokyo")

NameError: name 'AttractionsRecommendation' is not defined

# အဆင့် ၆: စမ်းသပ်မှုမှုလွှာ ၂ - ပါရီ ခရီးသွား အကြံပြုချက်များ


In [14]:
await display_travel_recommendations("Paris")

## အဆင့် ၇: ပုံမှန် တိုင်းတာခြင်း - တစ်ပြိုင်နက်လုပ်ခြင်း နှင့် ဆက်တိုက်လုပ်ခြင်း

တစ်ပြိုင်နက်လုပ်ခြင်းနှင့် ဆက်တိုက်လုပ်ခြင်းတို့အကြား လုပ်ဆောင်မှုကွာခြားချက်ကို တိုင်းတာပြီး တစ်ပြိုင်နက် ကြီးကြပ်ခြင်း၏ အကျိုးကျေးဇူးများကို ပြသကြရအောင်။


In [15]:
import time
from agent_framework import SequentialBuilder


async def measure_concurrent_performance(destination: str):
    """Measure concurrent execution time."""
    start_time = time.time()

    events = await workflow.run(f"I want travel recommendations for {destination}")
    outputs = events.get_outputs()

    end_time = time.time()
    return end_time - start_time, len(outputs[0]) if outputs else 0


async def measure_sequential_performance(destination: str):
    """Measure sequential execution time."""
    # Build sequential workflow for comparison
    sequential_workflow = (
        SequentialBuilder()
        .participants([attractions_agent, dining_agent, history_agent])
        .build()
    )
    start_time = time.time()

    events = await sequential_workflow.run(f"I want travel recommendations for {destination}")
    outputs = events.get_outputs()

    end_time = time.time()
    return end_time - start_time, len(outputs[0]) if outputs else 0


async def performance_comparison():
    """Compare concurrent vs sequential performance."""
    test_destination = "Barcelona"

    display(HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>Performance Comparison Test</h3>
        <p style='margin: 0;'>Testing with destination: <strong>Barcelona</strong></p>
    </div>
    """))

    # Test concurrent execution
    print("Running concurrent workflow...")
    concurrent_time, concurrent_msgs = await measure_concurrent_performance(test_destination)

# Test sequential execution
    print("Running sequential workflow...")
    sequential_time, sequential_msgs = await measure_sequential_performance(test_destination)

    # Calculate performance improvement
    improvement = ((sequential_time - concurrent_time) / sequential_time) * 100

    display(HTML(f"""
    <div style='padding: 25px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 12px; 
                box-shadow: 0 4px 12px rgba(102,126,234,0.4); margin: 20px 0;'>
        <h2 style='margin: 0 0 20px 0;'>Performance Results</h2>
        <div style='display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin-bottom: 20px;'>
            <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;'>
                <h4 style='margin: 0 0 10px 0;'>⚡ Concurrent Execution</h4>
                <p style='margin: 0; font-size: 24px; font-weight: bold;'>{concurrent_time:.2f}s</p>
                <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>{concurrent_msgs} messages</p>
            </div>
            <div style='background: rgba(255,255,255,0.1); padding: 15px; border-radius: 8px;'>
                <h4 style='margin: 0 0 10px 0;'>🔄 Sequential Execution</h4>
                <p style='margin: 0; font-size: 24px; font-weight: bold;'>{sequential_time:.2f}s</p>
                <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>{sequential_msgs} messages</p>
            </div>
        </div>
        <div style='background: rgba(255,255,255,0.15); padding: 15px; border-radius: 8px;'>
            <h4 style='margin: 0 0 10px 0;'>Performance Improvement</h4>
            <p style='margin: 0; font-size: 20px; font-weight: bold;'>{improvement:.1f}% faster</p>
            <p style='margin: 5px 0 0 0; font-size: 14px; opacity: 0.9;'>
                Saved {sequential_time - concurrent_time:.2f} seconds with concurrent execution
            </p>
        </div>
    </div>
    """))
# Run performance comparison
await performance_comparison()

Running concurrent workflow...
Running sequential workflow...


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ေၾကညာခ်က္**:
ဤစာတမ်းကို AI ဘာသာပြန်ခြင်းဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ဖြင့် ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးပမ်းနေသော်လည်း၊ စက်ရုပ်အလိုအလျောက် ဘာသာပြန်ချက်များတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ရှိနိုင်ကြောင်း သတိပြုပါ။ မူရင်းစာတမ်းကို ထိုဘာသာစကားဖြင့်သာ အာဏာရှိသည့် အချက်အလက် အဖြစ် ယူဆပါရန် လိုအပ်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် မူလကျွမ်းကျင်သော လူသား ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ချက်ကို အသုံးပြုမှုကြောင့် ဖြစ်ပေါ်လာနိုင်သည့် စကားမမှန်ခြင်း သို့မဟုတ် နားမလည်မှုများအတွက် ကျွန်ုပ်တို့မှာ တာဝန်မယူပါ။
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
